In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping


2025-03-10 03:02:08.446421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')

target_treino = data_treino.pop('rainfall')


# Carregar as previsões dos arquivos CSV
df_lr_predictions = pd.read_csv('/home/caio/github/k-3/meta_features/lr_final.csv')
df_catboost_predictions = pd.read_csv('/home/caio/github/k-3/meta_features/cat_final.csv')
df_neutal_predictions = pd.read_csv('/home/caio/github/k-3/meta_features/neural_final.csv')


df_lr_predictions_teste = pd.read_csv('/home/caio/github/k-3/meta_features/lr_treino.csv')
df_catboost_predictions_teste = pd.read_csv('/home/caio/github/k-3/meta_features/cat_treino.csv')
df_neural_predictions_teste = pd.read_csv('/home/caio/github/k-3/meta_features/neural_teste.csv')


# Empilhar as previsões dos modelos base em um novo conjunto de dados
meta_features_previsao = pd.concat([df_lr_predictions, df_catboost_predictions, df_neutal_predictions], axis=1)
meta_features = pd.concat([df_lr_predictions_teste, df_catboost_predictions_teste, df_neural_predictions_teste], axis=1)






In [3]:
X_train, X_val, y_train, y_val = train_test_split(meta_features, target_treino, test_size=0.2, random_state=42)

model_meta = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Adicionando camada de Dropout
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer (regressão)
])

2025-03-10 03:02:10.306428: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)



# Compilar o modelo
model_meta.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Treinar o meta-modelo

model_meta.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])



Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 0.1733 - val_loss: 0.0900
Epoch 2/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0764 - val_loss: 0.0727
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0677 - val_loss: 0.0670
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0601 - val_loss: 0.0634
Epoch 5/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0587 - val_loss: 0.0631
Epoch 6/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0576 - val_loss: 0.0624
Epoch 7/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0559 - val_loss: 0.0643
Epoch 8/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0542 - val_loss: 0.0597
Epoch 9/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0538 - val_loss: 0.0601
Epoch 10/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0524 - val_loss: 0.0575
Epoch 11

In [5]:
previsao = model_meta.predict(meta_features_previsao)

23/23 [==============================] - 0s 4ms/step


In [6]:
caminho_destino = 'previsoes/linear/Meta_v1.csv'
resultado = pd.DataFrame({
    'id': data_teste['id'],
    'rainfall': previsao.flatten() 
})

resultado.to_csv(caminho_destino, index=False)